In [1]:
import pandas as pd
import numpy as np
import os
from importlib import reload
os.chdir("/Users/bytedance/MLTrader-deploy/")
from research.strategies.automl_strategies import *
from research.strategies import strategy_utils
reload(strategy_utils)
DATA_ROOT = "/Users/bytedance/binance_data/csv_data"
bar_window = 240
h5_path = os.path.join(DATA_ROOT, f"dataview_{bar_window}m.h5")
keys = ['Close', 'High', 'Low', 'Open', 'Volume', 'Count', 'timestamp', 'Asset_ID']

/opt/miniconda3/envs/vnpy3.9/lib/python3.9/site-packages/backtesting/_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

In [2]:
adx_window = 14
long_volume_period = 20
left = 6
right = 3

In [3]:
global_dv = {}
for key in keys:
    global_dv[key] = pd.read_hdf(h5_path, key)
    global_dv[key].index = pd.to_datetime(global_dv[key].index, unit="s")

In [4]:
live_days = {}
all_symbols = set(global_dv["Close"].columns)
for symbol in all_symbols:
    symbol_index = global_dv["Close"][symbol].dropna().index
    live_days[symbol] = (symbol_index[-1] - symbol_index[0]).days

In [5]:
live_days = pd.Series(live_days)
live_days.sort_values()

HFTUSDT       29
APTUSDT       48
GMXUSDT       62
LUNCUSDT      88
OPUSDT       188
            ... 
ADAUSDT     1694
LTCUSDT     1819
BNBUSDT     1856
ETHUSDT     1937
BTCUSDT     1937
Length: 89, dtype: int64

In [6]:
short_life = set(live_days[live_days < 180].index)
print("short life coins (小于180天): ", short_life)
all_symbols = all_symbols - short_life

short life coins (小于180天):  {'LUNCUSDT', 'APTUSDT', 'HFTUSDT', 'GMXUSDT'}


In [7]:
" ".join(all_symbols)

'LUNAUSDT CELOUSDT CHZUSDT ROSEUSDT MANAUSDT SNXUSDT SOLUSDT RUNEUSDT SUSHIUSDT SHIBUSDT FTMUSDT ETCUSDT DOTUSDT MATICUSDT VETUSDT DASHUSDT AXSUSDT WAVESUSDT KAVAUSDT REEFUSDT GRTUSDT OPUSDT C98USDT BURGERUSDT TRXUSDT ARUSDT AAVEUSDT SANTOSUSDT GALUSDT UNIUSDT TWTUSDT LDOUSDT XRPUSDT BCHUSDT MDXUSDT YFIUSDT ADAUSDT AUDUSDT CAKEUSDT NEARUSDT ETHUSDT PEOPLEUSDT EGLDUSDT BNXUSDT VGXUSDT XMRUSDT GMTUSDT LRCUSDT CRVUSDT XLMUSDT GALAUSDT INJUSDT BTCUSDT YFIIUSDT BANDUSDT FILUSDT BNBUSDT ALICEUSDT SXPUSDT DOGEUSDT DYDXUSDT SANDUSDT FLOWUSDT VIDTUSDT PORTOUSDT AVAXUSDT GBPUSDT UNFIUSDT ZILUSDT ENSUSDT ATOMUSDT LTCUSDT THETAUSDT ICPUSDT MASKUSDT EOSUSDT LINKUSDT KLAYUSDT QNTUSDT SFPUSDT LAZIOUSDT JASMYUSDT APEUSDT ALGOUSDT LITUSDT'

In [8]:
for key in keys:
    global_dv[key] = global_dv[key][list(all_symbols)]

In [9]:
features = {}
features["adx"] = pd.DataFrame(columns=global_dv["Close"].columns, index=global_dv["Close"].index)
features["minus"] = pd.DataFrame(columns=global_dv["Close"].columns, index=global_dv["Close"].index)
features["plus"] = pd.DataFrame(columns=global_dv["Close"].columns, index=global_dv["Close"].index)
for symbol in all_symbols:
    h = global_dv["High"][symbol].values
    l = global_dv["Low"][symbol].values
    c = global_dv["Close"][symbol].values
    
    plus, minus, adx = calcADX(h, l, c, adx_window)
    features["adx"][symbol] = adx
    features["minus"][symbol] = minus
    features["plus"][symbol] = plus

In [10]:
features["slow_ma"] = global_dv["Close"].rolling(20).mean()
features["fast_ma"] = global_dv["Close"].rolling(5).mean()
features["ma_diff"] = features["fast_ma"] - features["slow_ma"]

In [49]:
from jaqs_fxdayu.research import SignalDigger
from jaqs_fxdayu.research.signaldigger.multi_factor import get_factors_ic_df
from jaqs_fxdayu.research.signaldigger.analysis import ic_stats
from jaqs_fxdayu.research.signaldigger.analysis import return_stats

In [38]:
sd = SignalDigger()
sd.process_signal_before_analysis(signal=features["ma_diff"],
                                  price=global_dv["Close"],
                                  high=global_dv["High"],
                                  low=global_dv["Low"],
                                  n_quantiles=10,
                                  period=5,
                                  )

/opt/miniconda3/envs/vnpy3.9/lib/python3.9/site-packages/jaqs/util/numeric.py:16: RuntimeWarning: divide by zero encountered in divide
  res = np.floor(rank / divisor) + 1.0
/opt/miniconda3/envs/vnpy3.9/lib/python3.9/site-packages/jaqs/util/numeric.py:16: RuntimeWarning: invalid value encountered in divide
  res = np.floor(rank / divisor) + 1.0


Nan Data Count (should be zero) : 0;  Percentage of effective data: 46%


In [48]:
ic_stats(sd.signal_data)

/opt/miniconda3/envs/vnpy3.9/lib/python3.9/site-packages/scipy/stats/_stats_py.py:4881: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  warnings.warn(stats.ConstantInputWarning(warn_msg))


,return_ic,upside_ret_ic,downside_ret_ic
IC Mean,0.010609,5.221520e-02,-3.236851e-02
IC Std.,0.350656,3.623334e-01,3.696229e-01
t-stat(IC),3.247351,1.552226e+01,-9.432574e+00
p-value(IC),0.001168,8.511179e-54,4.759727e-21
IC Skew,0.171144,-3.724186e-02,1.666091e-01
IC Kurtosis,1.706901,1.375929e+00,1.345156e+00
Ann. IR,0.030254,1.441081e-01,-8.757170e-02


In [51]:
return_stats(sd.signal_data, is_event=False, period=5)

,long_ret,short_ret,long_short_ret,top_quantile_ret,bottom_quantile_ret,tmb_ret,all_sample_ret
t-stat,4.252495,1.610441,-2.229960,2.817781,2.233135,-2.233151,2.234465
p-value,0.000020,0.107330,0.025770,0.004840,0.025540,0.025560,0.025450
skewness,3.133972,-3.793805,-48.332152,3.661971,101.310624,-48.333131,301.436085
kurtosis,69.082965,82.462934,2340.411055,77.904824,10290.296459,2340.578374,91113.502264
Ann. Ret,0.110889,0.037577,-24.488464,0.050636,860.428643,-419.572879,97.279314
Ann. Vol,0.403712,0.361250,170.015879,0.518389,12497.816199,2908.802636,4201.158096
Ann. IR,0.274675,0.104021,-0.144036,0.097680,0.068846,-0.144242,0.023155
occurance,11602.000000,11602.000000,11602.000000,40277.000000,50924.000000,11602.000000,450703.000000


In [1]:
# load各个周期的策略盈利能力

In [3]:
stra_path = "/Users/bytedance/MLTrader-deploy/.vscode/MaximumV2"
all_stra_cls = os.listdir(stra_path)

In [128]:
equity_dfs = {}
for folder in all_stra_cls:
    if folder == ".DS_Store":
        continue
    symbol, period, stra_name, _ = folder.split("_")
    period = int(period.replace("m", ""))
    equity_path = os.path.join(stra_path, folder, "0", f"{stra_name}_{symbol}_equity.csv")
    metrics_path = os.path.join(stra_path, folder, "0", f"{stra_name}_{symbol}_result.json")
    equity_df = pd.read_csv(equity_path, index_col=0)
    result_df = pd.read_csv(metrics_path, index_col=0)
    equity_dfs.setdefault(symbol, {})
    equity_dfs[symbol].setdefault(period, {})
    equity_dfs[symbol][period] = (equity_df, result_df)

In [137]:
data = {}
p = figure(toolbar_location="above")
# for period in [15, 60, 240]:
period = 15
for symbol in all_symbols:
    equity = equity_dfs[symbol][period][0]["Equity"]
    data[symbol] = equity
data_df = pd.DataFrame(data)
# data_df.plot(figsize=(40, 20))

In [138]:
data = {}
p = figure(toolbar_location="above")
# for period in [15, 60, 240]:
period = 60
for symbol in all_symbols:
    equity = equity_dfs[symbol][period][0]["Equity"]
    data[symbol] = equity
data_df = pd.DataFrame(data)
# data_df.plot(figsize=(40, 20))

In [139]:
data = {}
p = figure(toolbar_location="above")
# for period in [15, 60, 240]:
period = 240
for symbol in all_symbols:
    equity = equity_dfs[symbol][period][0]["Equity"]
    data[symbol] = equity
data_df = pd.DataFrame(data)
# data_df.plot(figsize=(40, 20))


In [114]:
data_df.head(3)

,SNXUSDT,FTMUSDT,SUSHIUSDT,ZILUSDT,LINKUSDT,KAVAUSDT,SANDUSDT,VGXUSDT,WAVESUSDT,LDOUSDT,...,MDXUSDT,DYDXUSDT,ROSEUSDT,ALICEUSDT,GALUSDT,YFIIUSDT,BNBUSDT,CELOUSDT,AAVEUSDT,ENSUSDT
2020-01-01 04:00:00,NaN,10000000.0,NaN,10000000.0,10000000.0,10000000.0,NaN,NaN,10000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10000000.0,NaN,NaN,NaN
2020-01-01 08:00:00,NaN,10000000.0,NaN,10000000.0,10000000.0,10000000.0,NaN,NaN,10000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10000000.0,NaN,NaN,NaN
2020-01-01 12:00:00,NaN,10000000.0,NaN,10000000.0,10000000.0,10000000.0,NaN,NaN,10000000.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,10000000.0,NaN,NaN,NaN


In [177]:
data = {}
p = figure(toolbar_location="above")
# for period in [15, 60, 240]:
period = 60
for symbol in all_symbols:
    equity = equity_dfs[symbol][period][1]["0"]
    data[symbol] = equity
data_df = pd.DataFrame(data)
data_df = data_df.T
# data_df.plot(figsize=(40, 20))
data_df["Sharpe Ratio"] = data_df["Sharpe Ratio"].astype(float)
filter_data_df = data_df[(data_df["Sharpe Ratio"] > 0.3)]

In [178]:
filter_data_df

,Start,End,Duration,Exposure Time [%],Equity Final [$],Equity Peak [$],Return [%],Buy & Hold Return [%],Return (Ann.) [%],Volatility (Ann.) [%],...,Profit Factor (short),Expectancy [%],Expectancy [%](long),Expectancy [%](short),SQN,_strategy,_equity_curve,_trades,alpha [%],beta [%]
SHIBUSDT,2021-05-10 12:00:00,2022-12-07 00:00:00,575 days 12:00:00,49.25794541374068,81150810.48380594,199467184.27051064,711.5081048380594,-68.66374957926624,276.01276619417126,798.3924363031026,...,1.3779207887752485,1.2882248956522002,2.299692911624536,0.647797910664691,0.5761630266063501,MaximalistV2,Equity DrawdownPct...,Size EntryBar ExitBar EntryP...,0.0217,19.2686
FILUSDT,2020-10-15 18:00:00,2022-12-07 00:00:00,782 days 06:00:00,49.34753661784288,42702699.90363656,104143850.38694626,327.02699903636557,-93.08964937874592,96.56923951756546,289.02923193341667,...,1.266680213596402,0.5219175599201324,0.6341791458289414,0.45313757418727063,0.5384625782577152,MaximalistV2,Equity DrawdownPct...,Size EntryBar ExitBar EntryPrice ...,0.0131,11.6219
EGLDUSDT,2020-09-03 04:00:00,2022-12-07 00:00:00,824 days 20:00:00,47.537505682679196,46884650.992437705,86991525.14754745,368.846509924377,92.08967851099828,97.9341928189987,248.11646574319178,...,1.026843333841046,0.6667514630201876,1.705871130625334,0.062046372173045594,0.8398882793029208,MaximalistV2,Equity DrawdownPct...,Size EntryBar ExitBar EntryPrice E...,0.0125,7.1495
PEOPLEUSDT,2021-12-23 07:00:00,2022-12-07 00:00:00,348 days 17:00:00,41.373954599761056,26996053.12143213,52756009.417328805,169.9605312143213,-83.54407294832826,181.69852404441394,562.6664038931415,...,1.641894797878225,0.802140068270372,0.3787135624057131,1.084424405513478,0.656388099444317,MaximalistV2,Equity DrawdownPct...,Size EntryBar ExitBar EntryPrice...,0.0201,20.3039
RUNEUSDT,2020-09-04 07:00:00,2022-12-07 00:00:00,823 days 17:00:00,46.90945877592312,337637620.0898674,408167729.577136,3276.3762008986737,99.59735404083978,374.4863254594531,673.9748060692767,...,1.4601082945348394,0.9245498811890922,1.035784521162081,0.8539520446737278,1.7594680762964852,MaximalistV2,Equity DrawdownPct...,Size EntryBar ExitBar EntryPrice ...,0.0231,9.9457
ETHUSDT,2020-01-01 01:00:00,2022-12-07 00:00:00,1070 days 23:00:00,53.86321195144724,107887747.44402124,116261261.41385104,978.8774744402124,886.5135407775275,124.75615369474382,175.54670308404604,...,1.0590950079615145,0.489032182146264,1.0950335051297946,0.1047757041642257,1.7803228160238624,MaximalistV2,Equity DrawdownPct...,Size EntryBar ExitBar EntryPrice Exi...,0.0124,-2.3231


In [146]:
data_df.sort_values(by="Sharpe Ratio", axis=1, ascending=False)

,ETHUSDT,RUNEUSDT,EGLDUSDT,SHIBUSDT,FILUSDT,PEOPLEUSDT,GALAUSDT,AXSUSDT,FTMUSDT,ETCUSDT,...,THETAUSDT,ATOMUSDT,CHZUSDT,TWTUSDT,BANDUSDT,PORTOUSDT,UNIUSDT,BURGERUSDT,LAZIOUSDT,ENSUSDT
Start,2020-01-01 01:00:00,2020-09-04 07:00:00,2020-09-03 04:00:00,2021-05-10 12:00:00,2020-10-15 18:00:00,2021-12-23 07:00:00,2021-09-13 07:00:00,2020-11-04 14:00:00,2020-01-01 01:00:00,2020-01-01 01:00:00,...,2020-01-01 01:00:00,2020-01-01 01:00:00,2020-01-01 01:00:00,2021-01-27 09:00:00,2020-01-01 01:00:00,2021-11-16 13:00:00,2020-09-17 04:00:00,2021-04-30 13:00:00,2021-10-21 13:00:00,2021-11-10 08:00:00
End,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,...,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00,2022-12-07 00:00:00
Duration,1070 days 23:00:00,823 days 17:00:00,824 days 20:00:00,575 days 12:00:00,782 days 06:00:00,348 days 17:00:00,449 days 17:00:00,762 days 10:00:00,1070 days 23:00:00,1070 days 23:00:00,...,1070 days 23:00:00,1070 days 23:00:00,1070 days 23:00:00,678 days 15:00:00,1070 days 23:00:00,385 days 11:00:00,810 days 20:00:00,585 days 11:00:00,411 days 11:00:00,391 days 16:00:00
Exposure Time [%],53.86321195144724,46.90945877592312,47.537505682679196,49.25794541374068,49.34753661784288,41.373954599761056,41.37483787289234,44.220995682824196,45.45985060690943,48.71226268285092,...,46.245720510426395,47.097727980080926,48.895113600995955,46.49435166994106,47.08216619981326,44.498486813661906,49.01598067930733,40.976373469968685,38.58849736735521,44.314434634613335
Equity Final [$],107887747.44402124,337637620.0898674,46884650.992437705,81150810.48380594,42702699.90363656,26996053.12143213,37631877.00664417,130192190.97036216,74339283.99967447,32018504.327830024,...,1897122.859398531,6236462.313579268,1281996.5351761256,976087.996116866,2141193.1396673354,5563698.3055351935,7530373.711253703,441279.8292970454,2372817.818532142,523603.77608002914
Equity Peak [$],116261261.41385104,408167729.577136,86991525.14754745,199467184.27051064,104143850.38694626,52756009.417328805,103840120.81757373,182423670.8049707,239090692.95008093,48497915.51247056,...,34206022.302771196,18435847.593612153,14636838.103785481,16361039.166709904,49686606.486303166,13883289.530966004,31904855.186301418,17479762.453750003,11752338.993584007,10221870.20727
Return [%],978.8774744402124,3276.3762008986737,368.846509924377,711.5081048380594,327.02699903636557,169.9605312143213,276.31877006644163,1201.9219097036216,643.3928399967447,220.1850432783002,...,-81.02877140601468,-37.63537686420732,-87.18003464823873,-90.23912003883135,-78.58806860332665,-44.363016944648066,-24.696262887462975,-95.58720170702955,-76.27182181467859,-94.7639622391997
Buy & Hold Return [%],886.5135407775275,99.59735404083978,92.08967851099828,-68.66374957926624,-93.08964937874592,-83.54407294832826,-58.8414170594496,5064.180864233921,2170.718232044199,335.30538175502664,...,939.96314637798,141.01469890943577,2187.377537980974,710.1454293628809,817.7958446251129,-66.8611466090865,105.5067837190742,-95.00392156862745,-75.65492690472874,-76.25737079431148
Return (Ann.) [%],124.75615369474382,374.4863254594531,97.9341928189987,276.01276619417126,96.56923951756546,181.69852404441394,192.2837349981681,240.79365013640856,97.98708114702102,48.62145583153523,...,-43.21909923477926,-14.850999877052862,-50.312312135296146,-71.31906236189775,-40.83044475870321,-42.4773310998689,-11.970523370499354,-85.63600111550295,-71.95387331193763,-93.53944618928651
Volatility (Ann.) [%],175.54670308404604,673.9748060692767,248.11646574319178,798.3924363031026,289.02923193341667,562.6664038931415,695.8244985625734,895.9691834289071,391.4453172536009,200.47463687746864,...,73.55528159024871,92.1781547852492,83.16004933859621,42.74640

In [116]:
data_df.index.name = "datetime"

In [127]:
# dates = np.array(data_df.index, dtype=np.datetime64)
# source = ColumnDataSource(data_df)

# p = figure(height=300, width=800, tools="xpan", toolbar_location=None,
#            x_axis_type="datetime", x_axis_location="above",
#            background_fill_color="#efefef", x_range=(dates[1500], dates[2500]))

# p.line('datetime', 'SNXUSDT', source=source)
# p.yaxis.axis_label = 'Equity'

# # select = figure(title="Drag the middle and edges of the selection box to change the range above",
# #                 height=130, width=800, y_range=p.y_range,
# #                 x_axis_type="datetime", y_axis_type=None,
# #                 tools="", toolbar_location=None, background_fill_color="#efefef")

# # range_tool = RangeTool(x_range=p.x_range)
# # range_tool.overlay.fill_color = "navy"
# # range_tool.overlay.fill_alpha = 0.2
# save(p, "test.html")

/opt/miniconda3/envs/vnpy3.9/lib/python3.9/site-packages/bokeh/io/saving.py:142: UserWarning: save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN
  warn("save() called but no resources were supplied and output_file(...) was never called, defaulting to resources.CDN")
/opt/miniconda3/envs/vnpy3.9/lib/python3.9/site-packages/bokeh/io/saving.py:154: UserWarning: save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'
  warn("save() called but no title was supplied and output_file(...) was never called, using default title 'Bokeh Plot'")
ERROR:bokeh.core.validation.check:E-1001 (BAD_COLUMN_NAME): Glyph refers to nonexistent column name. This could either be due to a misspelling or typo, or due to an expected column being missing. : key "x" value "datetime" [renderer: GlyphRenderer(id='8349', ...)]


'/Users/bytedance/MLTrader-deploy/test.html'

In [34]:
data = {}
p = figure(toolbar_location="above")
# for period in [15, 60, 240]:
period = 60
for symbol in all_symbols:
    equity = equity_dfs[symbol][period]["Equity"]
    data[symbol] = equity
data_df = pd.DataFrame(data)

,SNXUSDT,FTMUSDT,SUSHIUSDT,ZILUSDT,LINKUSDT,KAVAUSDT,SANDUSDT,VGXUSDT,WAVESUSDT,LDOUSDT,...,MDXUSDT,DYDXUSDT,ROSEUSDT,ALICEUSDT,GALUSDT,YFIIUSDT,BNBUSDT,CELOUSDT,AAVEUSDT,ENSUSDT
2020-01-01 04:00:00,NaN,1.000000e+07,NaN,1.000000e+07,1.000000e+07,1.000000e+07,NaN,NaN,1.000000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+07,NaN,NaN,NaN
2020-01-01 08:00:00,NaN,1.000000e+07,NaN,1.000000e+07,1.000000e+07,1.000000e+07,NaN,NaN,1.000000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+07,NaN,NaN,NaN
2020-01-01 12:00:00,NaN,1.000000e+07,NaN,1.000000e+07,1.000000e+07,1.000000e+07,NaN,NaN,1.000000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+07,NaN,NaN,NaN
2020-01-01 16:00:00,NaN,1.000000e+07,NaN,1.000000e+07,1.000000e+07,1.000000e+07,NaN,NaN,1.000000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+07,NaN,NaN,NaN
2020-01-01 20:00:00,NaN,1.000000e+07,NaN,1.000000e+07,1.000000e+07,1.000000e+07,NaN,NaN,1.000000e+07,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+07,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-12-06 08:00:00,5.994065e+06,3.712515e+08,3.771356e+06,1.263687e+08,1.257556e+07,2.836197e+07,1.323039e+08,6.314531e+06,4.589570e+07,1.296472e+07,...,3.266753e+06,7.749131e+06,9.584829e+07,3.086099e+06,6.684238e+06,1.404080e+07,4.916244e+07,4.090242e+06,1.717151e+07,6.801921e+06
2022-12-06 12:00:00,5.920775e+06,3.712515e+08,3.771356e+06,1.263687e+08,1.257556e+07,2.836197e+07,1.311462e+08,5.923089e+06,4.543536e+07,1.296472e+07,...,3.266753e+06,7.749131e+06,9.584829e+07,3.086099e+06,6.684238e+06,1.344068e+07,4.916244e+07,4.021329e+06,1.717151e+07,6.801921e+06
2022-12-06 16:00:00,5.887184e+06,3.712515e+08,3.771356e+06,1.263687e+08,1.257556e+07,2.836197e+07,1.328185e+08,5.923089e+06,4.617190e+07,1.296472e+07,...,3.266753e+06,7.749131e+06,9.584829e+07,3.086099e+06,6.684238e+06,1.329609e+07,4.916244e+07,4.008800e+06,1.717151e+07,6.801921e+06
2022-12-06 20:00:00,5.862755e+06,3.712515e+08,3.841074e+06,1.263687e+08,1.257556e+07,2.836197e+07,1.319823e+08,5.923089e+06,4.589570e+07,1.296472e+07,...,3.246227e+06,7.749131e+06,9.584829e+07,3.086099e+06,6.684238e+06,1.390579e+07,4.916244e+07,3.977476e+06,1.717151e+07,6.801921e+06


In [ ]:
data_df_ret_portfolio = data_df.mean(axis=1)
data_df_ret_portfolio.plot()

In [72]:
price_max = global_dv["High"].rolling(4).max().shift(-4)
price_min = global_dv["Low"].rolling(4).min().shift(-4)
up_abs = np.abs(price_max - global_dv["Close"])
range_adv = price_max - price_min

atr = strategy_utils.get_atr(global_dv["High"], global_dv["Low"], global_dv["Close"], 20)
atr = pd.DataFrame(atr, index=global_dv["Close"].index, columns=global_dv["Close"].columns)

mae = up_abs / (range_adv + 1e-10)

In [101]:
mae.mean().mean()

0.500859742165005

In [107]:
# create event
global_dv["Ret1"] = global_dv["Close"].pct_change(1)
global_dv["Std1"] = global_dv["Ret1"].rolling(20).std()
f_volume = global_dv["Volume"] / global_dv["Volume"].rolling(20).mean() > 1.5
f_ret = global_dv["Close"] / global_dv["Std1"] > 1.5
ret = global_dv["Close"].pct_change(12)
ret_bool = ret.rank(axis=1, na_option="keep", pct=True) > 0.5

final_bool = f_volume & ret_bool & f_ret


In [108]:
f_volume.sum()

Asset_ID
AAVEUSDT      756
ADAUSDT      1557
ALGOUSDT     1148
ALICEUSDT     576
APEUSDT       251
             ... 
XMRUSDT      1234
XRPUSDT      1450
YFIIUSDT      789
YFIUSDT       807
ZILUSDT      1332
Length: 89, dtype: int64

In [109]:
mae[final_bool].describe()

Asset_ID,AAVEUSDT,ADAUSDT,ALGOUSDT,ALICEUSDT,APEUSDT,APTUSDT,ARUSDT,ATOMUSDT,AUDUSDT,AVAXUSDT,...,VETUSDT,VGXUSDT,VIDTUSDT,WAVESUSDT,XLMUSDT,XMRUSDT,XRPUSDT,YFIIUSDT,YFIUSDT,ZILUSDT
count,533.000000,1041.000000,768.000000,392.000000,168.000000,26.000000,373.000000,876.000000,478.000000,512.000000,...,326.000000,208.000000,265.000000,893.000000,971.000000,894.000000,978.000000,547.000000,552.000000,393.000000
mean,0.516923,0.527597,0.518845,0.485399,0.477929,0.538470,0.503259,0.507639,0.511863,0.514937,...,0.480221,0.499583,0.508278,0.468658,0.501524,0.499588,0.486412,0.521627,0.513134,0.516282
std,0.308664,0.309116,0.298460,0.295821,0.310858,0.280654,0.299268,0.296018,0.303202,0.292231,...,0.303643,0.309958,0.301760,0.304169,0.301287,0.304498,0.303607,0.314320,0.313106,0.304957
min,0.000000,0.000000,0.000000,0.000000,0.003606,0.083831,0.000000,0.002310,0.000000,0.001208,...,0.000913,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.227273,0.255866,0.257025,0.223266,0.189347,0.315572,0.268293,0.246567,0.231681,0.267463,...,0.205330,0.240914,0.238298,0.196439,0.233233,0.222826,0.205443,0.214602,0.208769,0.231750
50%,0.539417,0.523810,0.511753,0.493467,0.443867,0.536771,0.510848,0.502653,0.529865,0.541292,...,0.484180,0.465971,0.506494,0.441899,0.504644,0.500000,0.477323,0.554780,0.560289,0.524024
75%,0.796527,0.815574,0.788037,0.755514,0.768993,0.740272,0.745441,0.771663,0.781460,0.767713,...,0.740697,0.776688,0.793590,0.735465,0.770122,0.774712,0.767211,0.817714,0.803260,0.785425
max,1.000000,1.006467,1.066667,1.000000,0.999158,0.972000,1.001541,1.041379,1.013889,1.006667,...,1.000527,1.018868,1.002681,1.052045,1.002283,1.036613,1.000205,1.021505,1.005545,1.000000


In [110]:
mae[final_bool].mean().mean()

0.5042002182741981

In [90]:
mae.shape

(11627, 89)